### 2. Output in formato .xml 
Qui dobbiamo importare la [libreria per il parsing di XML](https://docs.python.org/3/library/xml.etree.elementtree.html). Se non è installata, dobbiamo eseguire da terminale il comando
`conda install lxml`, quindi chiudere e riavviare il notebook.

In [1]:
import xml.etree.ElementTree as ET

Vogliamo ottenere le seguenti informazioni, in analogia con il caso precedente:
- Specie atomiche
- Indici degli atomi
- Indici degli orbitali atomici
- Lista dei coefficienti $a_{\kappa l}$ per ogni livello elettronico $(nk)$ 

In [2]:
"""
Script to parse .xml output of projwfc.x.

Adapted from yambopy: https://github.com/yambo-code/yambopy
"""

import xml.etree.ElementTree as ET

#####################
# Input parameters  #
#####################

file_nm='data/atomic_proj.xml' #Can be relative or absolute path to file

########################
# End input parameters #
########################

datafile_xml = ET.parse(file_nm).getroot() #Read the .xml file

Ora troviamo alcuni parametri utilizzando le chiavi del file .xml.

In [3]:
"""
Script to parse .xml output of projwfc.x.

Adapted from yambopy: https://github.com/yambo-code/yambopy
"""

import xml.etree.ElementTree as ET

#####################
# Input parameters  #
#####################

file_nm='data/atomic_proj.xml' #Can be relative or absolute path to file

########################
# End input parameters #
########################

datafile_xml = ET.parse(file_nm).getroot() #Read the .xml file

# Loop over "subtrees"/fields
for field in datafile_xml: 
    
    # Select specific field and view its attributes as a dictionary
    if field.tag=="HEADER":
        # Number of kpoints
        N_kpoints = field.attrib["NUMBER_OF_K-POINTS"]
        # Number of bands
        N_bands = field.attrib["NUMBER_OF_BANDS"]
        # Number of atomic orbitals
        N_states = field.attrib["NUMBER_OF_ATOMIC_WFC"]

print(N_kpoints)
print(N_bands)
print(N_states)

25
22
162


Ora leggiamo i coefficienti $a_{\kappa l}$ per ogni stato elettronico $(nk)$.

In [4]:
"""
Script to parse .xml output of projwfc.x.

Adapted from yambopy: https://github.com/yambo-code/yambopy
"""

import xml.etree.ElementTree as ET
import numpy as np

#####################
# Input parameters  #
#####################

file_nm='data/atomic_proj.xml' #Can be relative or absolute path to file

########################
# End input parameters #
########################

datafile_xml = ET.parse(file_nm).getroot() #Read the .xml file

# Loop over "subtrees"/fields
for field in datafile_xml: 
    
    # Select specific field and view its attributes as a dictionary
    if field.tag=="HEADER":
        # Number of kpoints
        N_kpoints = int( field.attrib["NUMBER_OF_K-POINTS"] ) #Convert to int values read as strings
        # Number of bands
        N_bands   = int( field.attrib["NUMBER_OF_BANDS"] )
        # Number of atomic orbitals
        N_states  = int( field.attrib["NUMBER_OF_ATOMIC_WFC"] )
        
    if field.tag=="EIGENSTATES":
         
        states_raw = []
        #for i in range(N_kpoints): print(field[3*(i+1)-1].tag)
        for i in range(N_kpoints):
            i_proj_counter = 3*(i+1)-1
            for ATOMIC_WFC in field[i_proj_counter]:
                state_TMP = ATOMIC_WFC.text # Read field contents
                states_raw.append( state_TMP ) 

print(states_raw[0])
print(len(states_raw))



  0.19616813059482638      -0.31960795754084931     
 -0.10444346014227733      -0.50532786188480949     
 -0.27619309377809603        4.0509253021372739E-002
  0.45957964473449298      -0.18651697375552712     
  0.23617775698429117       -7.5437460315312788E-002
 -0.22336145031863702      -0.33245287974073939     
  0.11227797874911578       -1.9396110405623057E-002
  -9.0056710792922809E-002  -1.9112684643399928E-002
   7.9440950082556870E-005  -9.0222971374089391E-005
  -1.8515684175621283E-005   3.2175415316637444E-004
  -1.7609894642846419E-003  -4.5917609376162184E-004
   1.2005418029923409E-003  -8.1557491763816034E-005
  -1.3537300959693461E-006   2.1635654145768087E-004
   2.6492433709668383E-004   1.6389900760361818E-003
   1.4799667092141239E-003  -6.4516105594062464E-006
  -9.4872584743677579E-004  -8.4985397720781168E-005
  -5.8120839131172212E-004   9.8859326070621927E-004
  -3.1202219841848980E-005   1.0094726252390463E-005
   6.1241646793430129E-004   1.53094445207042

Adesso manipoliamo i dati per convertirli in una forma utilizzabile (questa dovrebbe essere una funzione definita apposta).

In [42]:
# We have a list of strings:

# 1. Split each string. Now we have a list of lists which contain string elements
states_convert = [ state_TMP.split() for state_TMP in states_raw]

print(states_convert[0])

# 2. Convert each element of each list to floats
for state_TMP in states_convert:
    for i in range(len(state_TMP)): state_TMP[i]=float(state_TMP[i])

print(states_convert[0])

# 3. Convert to numpy array
states_convert = np.array(states_convert)

print(states_convert.shape)

# 4. Reshape with the correct dimensions
states_convert = states_convert.reshape(N_kpoints,N_states,N_bands,2)

print(states_convert.shape)

# 5. Pair together real and imaginary parts of orbital weights
states_convert = states_convert[:,:,:,0] + 1j*states_convert[:,:,:,1]

print(states_convert[0,0,0])
print(states_convert.shape)

# 6. Take modulus square of weights
orbital_weights = np.absolute(states_convert)**2

print(orbital_weights[0,0,0])

# 7. Swap axes so that weights[ik,ib]=projections
orbital_weights = orbital_weights.swapaxes(1,2)

print(orbital_weights.shape)

# 7. Sort in descending order
orbital_weights = np.sort(orbital_weights,axis=2)
orbital_weights = np.flip(orbital_weights,axis=2)

for ik in range(N_kpoints):
    for ib in range(N_bands):
        print(np.max(orbital_weights[ik,ib]))

['0.19616813059482638', '-0.31960795754084931', '-0.10444346014227733', '-0.50532786188480949', '-0.27619309377809603', '4.0509253021372739E-002', '0.45957964473449298', '-0.18651697375552712', '0.23617775698429117', '-7.5437460315312788E-002', '-0.22336145031863702', '-0.33245287974073939', '0.11227797874911578', '-1.9396110405623057E-002', '-9.0056710792922809E-002', '-1.9112684643399928E-002', '7.9440950082556870E-005', '-9.0222971374089391E-005', '-1.8515684175621283E-005', '3.2175415316637444E-004', '-1.7609894642846419E-003', '-4.5917609376162184E-004', '1.2005418029923409E-003', '-8.1557491763816034E-005', '-1.3537300959693461E-006', '2.1635654145768087E-004', '2.6492433709668383E-004', '1.6389900760361818E-003', '1.4799667092141239E-003', '-6.4516105594062464E-006', '-9.4872584743677579E-004', '-8.4985397720781168E-005', '-5.8120839131172212E-004', '9.8859326070621927E-004', '-3.1202219841848980E-005', '1.0094726252390463E-005', '6.1241646793430129E-004', '1.5309444520704272E-0

Possiamo anche _importare_ il nostro precedente codice -- Parsing_Files_1 -- come una libreria: in jupyter, possiamo far convertire il contenuto di una cella nel formato .py standard, e poi importarlo nella cella di un altro notebook.

In [44]:
"""
Script to parse .xml output of projwfc.x.

Adapted from yambopy: https://github.com/yambo-code/yambopy
"""

import xml.etree.ElementTree as ET
import Parsing_Files_1 as pf1 #<------------------------------------ IMPORTED (must be in the same folder)

def obtain_stuff_from_plain_text_file(plain_text_filenm):
    """
    Perform parsing operations requiring Parsing_Files_1 
    """
    #Variables defined globally
    global state_dicts
    global elements_list
    global orbitals_list
    global orbitals_labels
    
    # Concise form of the __main__ in Parsing_Files_1
    fil = open(plain_text_filenm, 'r')
    lines = fil.readlines()
    wfc_lines = pf1.get_line_numbers(lines)[3]
    state_dicts   = pf1.get_state_info(lines,wfc_lines)
    elements_list = pf1.get_elements(state_dicts)
    orbitals_list, orbitals_labels = pf1.get_orbitals(state_dicts)

#####################
# Input parameters  #
#####################

file_nm='data/atomic_proj.xml' #Can be relative or absolute path to file

########################
# End input parameters #
########################

datafile_xml = ET.parse(file_nm).getroot() #Read the .xml file

# Loop over "subtrees"/fields
for field in datafile_xml: 
    
    # Select specific field and view its attributes as a dictionary
    if field.tag=="HEADER":
        # Number of kpoints
        N_kpoints = int( field.attrib["NUMBER_OF_K-POINTS"] ) #Convert to int values read as strings
        # Number of bands
        N_bands   = int( field.attrib["NUMBER_OF_BANDS"] )
        # Number of atomic orbitals
        N_states  = int( field.attrib["NUMBER_OF_ATOMIC_WFC"] )
        
# Read atomic species and orbitals labels and obtain states dictionary
obtain_stuff_from_plain_text_file('data/projwfc.out')
N_atoms = state_dicts[N_states-1]['atomnum']+1
pf1.print_str(file_nm,N_kpoints,N_bands,N_states,N_atoms,elements_list,orbitals_labels)

print(state_dicts[21])

====== Reading output of projwfc.x ======
--- File name: data/atomic_proj.xml ---
   
- Number of kpoints: 25
- Number of bands: 22
- Number of atoms: 27
     with species present:  W  Mo  S
- Number of states: 162
     Angular momenta present:  s  p  d
   
{'atomnum': 2, 'kind_name': 'Mo', 'magnetic_number': 0, 'angular_momentum': 1}
